In [28]:
import requests
import io
import re

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

In [2]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [4]:
document = convert_pdf_to_txt('data/toc_headings.pdf')

In [10]:
lines = document.split('\n')

In [13]:
lines = [x for x in lines if x]

In [24]:
lines = [x for x in lines if '_' not in x]
lines = [x for x in lines if 'Updated Tables' not in x]
lines = [x.lstrip('\x0c') for x in lines if '(cid:' not in x]

In [40]:
for index, line in enumerate(lines):
    if not line.startswith('CAS') and not re.search('^\s*[0-9]', line):
        print(line)

Complete Table of Contents - Volumes One and Two
Volume One
General Information
Adducts of Hemoglobin
Tobacco Smoke
Disinfection By-Products
Personal Care and Consumer Product Chemicals and Metabolites
Flame Retardant Metabolites
Fungicides and Metabolites
Herbicides and Metabolites
Sulfonyl Urea Herbicides
Insect Repellent and Metabolites
Carbamate Pesticide Metabolites
Organochlorine Pesticide Metabolites
Organophosphorus Insecticides: Specific Metabolites
Organophosphorus Insecticides: Dialkyl Phosphate Metabolites
Pyrethroid Metabolites 
Metals and Metalloids
Perchlorate and Other Anions
Perfluoroalkyl and Polyfluoroalkyl Substances: Surfactants
Phthalate and Phthalate Alternative Metabolites
Phytoestrogens and Metabolites
Polycyclic Aromatic Hydrocarbon Metabolites
Heterocyclic Amines
Volatile Organic Compounds (VOCs)
Volatile Organic Compound (VOC) Metabolites
Appendices
Volume Two
Special Analysis of Pooled Samples for Select Chemicals
Organochlorine Pesticides and Metabolites (